# Using Pipeline class from Transformers Library


In [ ]:
import SEEhug as hug

#im = imread('myimage.png')
results = hug.allcheck(10)
#results

FileNotFoundError: [Errno 2] No such file or directory: 'modelfile.txt'

In [ ]:
help(hug.allcheck)

In [ ]:
import pathlib
from transformers import pipeline
from PIL import Image
import matplotlib.pylab as plt
import numpy as np

In [ ]:
from wrapper import list_of_models
model = list_of_models.get_models()

In [ ]:
models = list(model)
# models = models[0:5]

#### Extract all the images from our dataset and divide them into ground truths and unsegmented images.

In [ ]:
imagepath = pathlib.Path('./Images/')
images = imagepath.glob('*')
inferred = []
GT = []
for image in images:
    if "GT" in image.name:
        GT.append(image)

    elif "GT" not in image.name and "ipy" not in image.name:
        inferred.append(image)
GT_paths = sorted(GT)
input_paths = sorted(inferred)




# Test the generated mask with ground truth using SEE-Segment

In [ ]:
import sys
sys.path.append('/Users/ishasharma/Desktop/SEE Git/see-segment/')

In [ ]:
from see import Segment_Fitness as sf

In [ ]:
fitness_table = []
for image_path, GT_path in zip(input_paths, GT_paths):
    
    image = Image.open(image_path).convert("RGB")
    
    # NOTE TO DIRK, why does adding [:,:,0] change the fitness?
    GT_image = np.array(Image.open(GT_path).convert("RGB"))[:,:,0]
    
    # Create subplots for each model's segmented image
    fig, axs = plt.subplots(1, len(models))
    
    fitness = []
    # Iterate over the models and display segmented images
    
    for i, model_name in enumerate(models):
        # Initialize an image segmentation pipeline
        try:
            segmentation_pipeline = pipeline("image-segmentation", model=model_name)
            outputs = segmentation_pipeline(image)
            inferred_segmentation = outputs[0]['mask']

            # Display the segmented image in the corresponding subplot
            ax = axs[i] if len(models) > 1 else axs
            ax.imshow(inferred_segmentation)
            fit = sf.FitnessFunction(np.array(inferred_segmentation), GT_image)
            fitness.append(fit[0])
        except:
            fitness.append("NaN")
            print("Gave error")
    fitness_table.append(fitness)

        # Adjust the layout to avoid overlapping images
    fig.tight_layout()

        # Show the figure for the current image
    plt.show()
  

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(fitness_table, index = inferred, columns = models)
df.index.names = ["Images"]
pd.options.display.float_format = '{:.3f}'.format
df